# Chapter 3: Classification

## 3.1 MNIST

## 3.2 Training a Binary Classifier

## 3.3 Performance Measures

### 3.3.1 Measuring Accuracy Using Cross-Validation

### 3.3.2 Confusion Matrix

### 3.3.3 Precision and Recall

### 3.3.4 Precision/Recall Trade-Off

### 3.3.5 The ROC Curve

## 3.4 Multiclass Classification

## 3.5 Error Analysis

## 3.6 Multilabel Classification

## 3.7 Multioutput Classification

## 3.8 Exercises